In [60]:
import xmltodict
import string

In [61]:
filename = "/home/santi/workspace/data/pubmed/pubmedsample18n0001.xml"
with open(filename,"r") as fp:
    data = xmltodict.parse(fp.read())

In [92]:
printable = set(string.printable)
def get_data(input_file):
    with open(input_file,"r") as fp:
        data = xmltodict.parse(fp.read())
    return data

def get_text(input_data):
    text= ""
    if isinstance(input_data,dict):
        for key in input_data.keys():
            text += get_text(input_data[key]) 
    elif isinstance(input_data,list):
        for item in input_data:
            text += get_text(item)
    elif isinstance(input_data,str):
        return input_data+" "
    elif input_data:
        out_text = filter(lambda x: x in printable,input_data)
        return out_text+" "
    else:
        return ""
    return text

In [63]:
print(get_text(data["PubmedArticleSet"]["PubmedArticle"][0]).strip())

MEDLINE NLM 1 973217 1976 12 03 2002 11 13 Print Print 0095-3814 Print 3 1 1976 Fall Topics in health care financing Top Health Care Financ Hospital debt management and cost reimbursement. 69-81 Y Y Blume F R FR eng D016428 Journal Article United States Top Health Care Financ 7509107 0095-3814 IM D000066 N Accounting D004469 Y Economics, Hospital D006739 Y Hospital Administration D014481 N Geographic United States pubmed 1976 1 1 medline 1976 1 1 0 1 entrez 1976 1 1 0 0 ppublish pubmed 973217


In [67]:
data_list = [data,data]

In [64]:
from dask.distributed import Client

In [65]:
client = Client('10.1.10.108:8786')

distributed.nanny - WARNING - Worker process still alive after 47 seconds, killing
distributed.nanny - WARNING - Worker process 21176 was killed by signal 15


In [100]:
data_list = client.map(get_data,[filename,filename,filename])
data_list = client.gather(data_list)

In [101]:
text_data = client.map(get_text,data_list)
final_text_data = client.gather(text_data)

In [102]:
len(final_text_data)

3

In [104]:
final_text_data[2]

u'MEDLINE NLM 1 973217 1976 12 03 2002 11 13 Print Print 0095-3814 Print 3 1 1976 Fall Topics in health care financing Top Health Care Financ Hospital debt management and cost reimbursement. 69-81 Y Y Blume F R FR eng D016428 Journal Article United States Top Health Care Financ 7509107 0095-3814 IM D000066 N Accounting D004469 Y Economics, Hospital D006739 Y Hospital Administration D014481 N Geographic United States pubmed 1976 1 1 medline 1976 1 1 0 1 entrez 1976 1 1 0 0 ppublish pubmed 973217 MEDLINE NLM 1 1669026 1993 11 15 2008 11 21 Print Print 0377-8231 Print Anniv No Pt 1 1991 Bulletin et memoires de l\'Academie royale de medecine de Belgique Bull. Mem. Acad. R. Med. Belg. [150th Anniversary Celebration of the Royal Academy of Medicine of Belgium. Part 1. Bruxelles, 26-28 September 1991]. 1-191 fre D016423 Congresses D016424 Overall D019477 Portraits Clbration du CL Anniversaire de l\'Acadmie Royal de Mdecine de Belgique. Premire partie. Bruxelles, 26-28 septembre 1991. Belgium 